# Projeto de Big Data

## Análise de satisfação dos clientes

---

- ### Instalar todas as bibliotecas usadas

    Aconselho criar e ativar uma venv antes de rodar o código

In [ ]:
# !pip3 install pandas plotly nbformat openpyxl
# !pip3 install -U kaleido
# !pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

- ### Vinculo com o google sheets e criação dos arquivos em .xlsx

In [ ]:
import os.path
import openpyxl

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1TUNcZRjWbSVB5bw-547Tv0bm5JdKzbjjC4GEi2xw27E"
SAMPLE_RANGE_NAME = "A:M"

"""
1 - Antes de começar crie uma pasta chamada 'security' e adicione o arquivo de credenciais do Google Cloud.
2 - Renomeie o arquivo para credentials.json ou altere o nome do arquivo no código.
"""

creds = None
if os.path.exists("security\\token.json"):
  creds = Credentials.from_authorized_user_file("security\\token.json", SCOPES)
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "security\\credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  with open("security\\token.json", "w") as token:
    token.write(creds.to_json())

try:
  service = build("sheets", "v4", credentials=creds)

  sheet = service.spreadsheets()
  result = (
      sheet.values()
      .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
      .execute()
  )
  
  val = result['values']
  
  if not os.path.exists('archives'):
    os.mkdir('archives')
  
  # Criar planilha em excel com base na planilha do google sheets
  book = openpyxl.Workbook()
  page = book['Sheet']
  for linha in val:
      page.append(linha)
  
  book.save('archives\\acaliacaoClentes.xlsx')
  
except HttpError as err:
  print(err)


- ### Criação e exibição do DataFrame

In [ ]:
# importar o modulo pandas
import pandas as pd

avaliacaoClientes = pd.read_excel('archives\\acaliacaoClentes.xlsx')
display(avaliacaoClientes)

- ### Criação dos gráficos / Dashboards das perguntas de multipla escolha

In [ ]:
# Importação do plotly.express para a criação de gráficos do tipo Histograma
  # Instalar o nbformat para a utilização da versão mais atual: pip install nbformat
import plotly.express as px
df_perguntasObjetivas = avaliacaoClientes.drop(columns=['Carimbo de data/hora','Endereço de e-mail',
                                                'Nome completo','Oque você melhoraria na academia?'])
# Cria um grafico para cada coluna
for colunas in df_perguntasObjetivas.columns:
  grafico = px.histogram(df_perguntasObjetivas,x=colunas, color=colunas)
  grafico.show()
  display(df_perguntasObjetivas[colunas].value_counts(normalize=True).map('{:.2%}'.format))
  print('\n')
